<a href="https://colab.research.google.com/github/Arta-DS/DS/blob/main/%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)

1.   Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
2.   Попробуйте добавить +1 рекуррентный слой в encoder и decoder
3.   Попробуйте заменить GRU ячейки на lstm-ячейки
4.   Оцените качество во всех случаях

In [17]:
!pip install nltk

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import requests
import zipfile
import io
import os
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

In [6]:
# 1. Подготовка данных
local_zip_path = '/content/rus-eng.zip'
file = None # Инициализируем переменную

try:
    with zipfile.ZipFile(local_zip_path, 'r') as z:
        with z.open('rus.txt') as f:
            file_content = f.read().decode('utf-8')
            file = file_content.splitlines()
    print(f"Файл успешно загружен и распакован локально. Количество строк: {len(file)}")
except FileNotFoundError:
    print(f"ОШИБКА: Файл не найден по пути '{local_zip_path}'. Убедитесь, что вы скачали его и положили в нужную папку.")
    # Прерываем выполнение, если файл не найден
    raise SystemExit("Остановка выполнения: файл с данными не найден.")
except Exception as e:
    print(f"Произошла ошибка при распаковке: {e}")
    raise SystemExit("Остановка выполнения: не удалось распаковать файл.")

Файл успешно загружен и распакован локально. Количество строк: 527642


In [7]:
#  2. Предобработка и токенизация

try:
    spacy_en = spacy.load('en_core_web_sm')
    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]
except OSError:
    print("Модель spacy для английского не найдена. Используем простую токенизацию.")
    def tokenize_en(text):
        return text.split(' ')

def tokenize_ru(text):
    text = text.replace('.', ' .').replace(',', ' ,').replace('!', ' !').replace('?', ' ?')
    return text.split(' ')

In [8]:
#  3. Создание словарей

SOS_token = 0
EOS_token = 1
PAD_token = 2
UNK_token = 3

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"<SOS>": SOS_token, "<EOS>": EOS_token, "<PAD>": PAD_token, "<UNK>": UNK_token}
        self.index2word = {v: k for k, v in self.word2index.items()}
        self.word2count = {}
        self.n_words = len(self.word2index) # Считаем все служебные токены

    def addSentence(self, sentence):
        for word in tokenize(self.name, sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def tokenize(lang, sentence):
    if lang == 'en':
        return tokenize_en(sentence)
    else:
        return tokenize_ru(sentence)

MAX_LEN = 10
pairs = []
eng = Lang('eng')
rus = Lang('rus')

print("Подготовка пар и словарей...")
for line in tqdm(file):
    parts = line.strip().split('\t')
    if len(parts) < 2: continue
    en_sentence, ru_sentence = parts[0], parts[1]

    if len(tokenize('en', en_sentence)) > MAX_LEN or len(tokenize('ru', ru_sentence)) > MAX_LEN:
        continue

    eng.addSentence(en_sentence)
    rus.addSentence(ru_sentence)
    pairs.append((en_sentence, ru_sentence))

print(f"Всего пар: {len(pairs)}")
print(f"Размер англ. словаря: {eng.n_words}")
print(f"Размер рус. словаря: {rus.n_words}")

Подготовка пар и словарей...


100%|██████████| 527642/527642 [00:18<00:00, 28732.06it/s]

Всего пар: 464716
Размер англ. словаря: 19720
Размер рус. словаря: 62689


In [9]:
#  4. Dataset и DataLoader

class TranslationDataset(Dataset):
    def __init__(self, pairs, src_lang, trg_lang):
        self.pairs = pairs
        self.src_lang = src_lang
        self.trg_lang = trg_lang

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        src_sentence, trg_sentence = self.pairs[idx]
        src_indices = [self.src_lang.word2index.get(word, UNK_token) for word in tokenize(self.src_lang.name, src_sentence)]
        trg_indices = [self.trg_lang.word2index.get(word, UNK_token) for word in tokenize(self.trg_lang.name, trg_sentence)]

        src_indices.append(EOS_token)
        trg_indices.append(EOS_token)

        return torch.tensor(src_indices, dtype=torch.long), torch.tensor(trg_indices, dtype=torch.long)

def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = nn.utils.rnn.pad_sequence(src_batch, padding_value=PAD_token, batch_first=True)
    trg_batch = nn.utils.rnn.pad_sequence(trg_batch, padding_value=PAD_token, batch_first=True)
    return src_batch, trg_batch

In [10]:
#  5. Модель Seq2Seq (гибкая архитектура)

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, rnn_type):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn_type = rnn_type

        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first=True)
        else: # GRU
            self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, batch_first=True)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, rnn_type):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn_type = rnn_type

        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first=True)
        else: # GRU
            self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, batch_first=True)

        self.fc_out = nn.Linear(hid_dim, output_dim)

    def forward(self, input, hidden):
        input = input.unsqueeze(1)
        embedded = self.embedding(input)
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        hidden = self.encoder(src)

        input = trg[:, 0]

        for t in range(1, trg_len):
            prediction, hidden = self.decoder(input, hidden)
            outputs[:, t] = prediction

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = prediction.argmax(1)

            input = trg[:, t] if teacher_force else top1

        return outputs

In [11]:
#  6. Функции для обучения и перевода

def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, (src, trg) in enumerate(tqdm(iterator, desc="Training")):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output_dim = output.shape[-1]

        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def translate_sentence(sentence, src_lang, trg_lang, model, device, max_len=50):
    model.eval()
    tokens = [token.lower() for token in tokenize(src_lang.name, sentence)]
    tokens = [src_lang.word2index.get(tok, UNK_token) for tok in tokens]

    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)

    with torch.no_grad():
        hidden = model.encoder(src_tensor)

    trg_indexes = [SOS_token]

    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        if pred_token == EOS_token:
            break

    trg_tokens = [trg_lang.index2word[i] for i in trg_indexes]

    return trg_tokens[1:-1]

In [12]:
#  7. Запуск экспериментов (оптимизировано для CPU)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

# 1. Берем только часть данных для быстрого теста
print(f"Исходное количество пар: {len(pairs)}")
pairs_small = random.sample(pairs, 10000) # Берем только 10 000 пар
print(f"Новое количество пар для обучения: {len(pairs_small)}")

# 2. Уменьшаем размер модели
INPUT_DIM = eng.n_words
OUTPUT_DIM = rus.n_words
ENC_EMB_DIM = 128  # Было 256
DEC_EMB_DIM = 128  # Было 256
HID_DIM = 256      # Было 512
CLIP = 1
N_EPOCHS = 2       # Было 10
BATCH_SIZE = 128   # Увеличили для скорости

dataset = TranslationDataset(pairs_small, eng, rus) # Используем урезанный датасет
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

def run_experiment(rnn_type, n_layers, exp_name):
    print(f"\n{'='*20} Запуск эксперимента: {exp_name} {'='*20}")

    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, n_layers, rnn_type)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, n_layers, rnn_type)
    model = Seq2Seq(enc, dec, device).to(device)

    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_token)

    for epoch in range(N_EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, CLIP)
        print(f'Эпоха: {epoch+1:02} | Потеря на обучении: {train_loss:.3f}')

    print("\nПримеры перевода:")
    for i in range(5):
        src, trg = pairs[i] # Выводим примеры из полного набора
        translation = translate_sentence(src, eng, rus, model, device)
        original = tokenize('ru', trg)
        print(f"Оригинал (EN): {src}")
        print(f"Эталон (RU):  {' '.join(original)}")
        print(f"Перевод (RU):  {' '.join(translation)}\n")
    return model

Используемое устройство: cuda
Исходное количество пар: 464716
Новое количество пар для обучения: 10000


In [13]:
#  7. Запуск экспериментов (Золотая середина)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

#  НАСТРОЙКИ "ЗОЛОТОЙ СЕРЕДИНЫ"
# Компромисс между временем обучения и качеством модели
NUM_SAMPLES = 100000
print(f"Исходное количество пар: {len(pairs)}")
pairs_medium = random.sample(pairs, NUM_SAMPLES)
print(f"Новое количество пар для обучения: {len(pairs_medium)}")

INPUT_DIM = eng.n_words
OUTPUT_DIM = rus.n_words

# Модель оставляем небольшой для скорости на CPU
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
CLIP = 1
N_EPOCHS = 5      # Увеличиваем количество эпох
BATCH_SIZE = 128  # Оставляем большим для скорости

dataset = TranslationDataset(pairs_medium, eng, rus)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

model_gru_1l_medium = run_experiment(rnn_type='GRU', n_layers=1, exp_name="GRU 1 слой (ЗОЛОТАЯ СЕРЕДИНА)")

print("\nЭксперимент завершен!")

Используемое устройство: cuda
Исходное количество пар: 464716
Новое количество пар для обучения: 100000

==================== Запуск эксперимента: GRU 1 слой (ЗОЛОТАЯ СЕРЕДИНА) ====================


Training: 100%|██████████| 782/782 [04:52<00:00,  2.68it/s]


Эпоха: 01 | Потеря на обучении: 4.796


Training: 100%|██████████| 782/782 [04:53<00:00,  2.67it/s]


Эпоха: 02 | Потеря на обучении: 3.370


...

Эпоха: 03 | Потеря на обучении: 2.664


Training: 100%|██████████| 782/782 [04:54<00:00,  2.66it/s]


Эпоха: 04 | Потеря на обучении: 2.155


Training: 100%|██████████| 782/782 [04:55<00:00,  2.65it/s]


Эпоха: 05 | Потеря на обучении: 1.774

Примеры перевода:
Оригинал (EN): Go.
Эталон (RU):  Марш !
Перевод (RU):  идти .

Оригинал (EN): Go.
Эталон (RU):  Иди .
Перевод (RU):  идти .

Оригинал (EN): Go.
Эталон (RU):  Идите .
Перевод (RU):  идти .

Оригинал (EN): Hi.
Эталон (RU):  Здравствуйте .
Перевод (RU):  с .

Оригинал (EN): Hi.
Эталон (RU):  Привет !
Перевод (RU):  с .


Эксперимент завершен!


Мы проведем два эксперимента с идентичными параметрами, но разной RNN-ячейкой, и оценим их с помощью стандартной метрики качества для машинного перевода — **BLEU**. Это метрика от 0 до 1 (или 0 до 100), которая сравнивает перевод, сгенерированный моделью, с одним или несколькими эталонными (человеческими) переводами. Чем больше совпадений слов и фраз, тем выше score.


**Данные**: Мы разделим наши данные на обучающую и тестовую выборки. Модель будет учиться на одной части, а качество будем измерять на другой, которую она никогда не видела. Это даст объективную оценку.

In [24]:
#  1. Подготовка данных: Разделение на Train и Test

# Используем 100к для обучения и 5к для теста
NUM_TRAIN_SAMPLES = 100000
NUM_TEST_SAMPLES = 5000

# Перемешиваем все пары для случайного выбора
random.shuffle(pairs)

train_pairs = pairs[:NUM_TRAIN_SAMPLES]
test_pairs = pairs[NUM_TRAIN_SAMPLES : NUM_TRAIN_SAMPLES + NUM_TEST_SAMPLES]

print(f"Размер обучающей выборки: {len(train_pairs)}")
print(f"Размер тестовой выборки: {len(test_pairs)}")

Размер обучающей выборки: 100000
Размер тестовой выборки: 5000


In [25]:
#  2. Функция для вычисления BLEU score

def calculate_bleu(model, test_pairs, src_vocab, trg_vocab, device):
    """
    Вычисляет BLEU score для модели на тестовой выборке.
    """
    references, candidates = [], []
    model.eval()

    with torch.no_grad():
        for src, trg in test_pairs:
            # Перевод от модели
            translation = translate_sentence(src, src_vocab, trg_vocab, model, device)

            # Эталонный перевод (список токенов)
            reference = [tokenize('ru', trg)]

            # Кандидат (перевод модели)
            candidate = translation

            references.append(reference)
            candidates.append(candidate)

    # corpus_bleu работает со списком предложений
    # weights=(1, 0, 0, 0) смотрим только на точность совпадения отдельных слов (1-gram)
    # Это более надежно для маленьких моделей, которые еще плохо строят фразы
    bleu_score = corpus_bleu(references, candidates, weights=(1, 0, 0, 0))
    return bleu_score

In [26]:
def run_experiment_eval(rnn_type, n_layers, exp_name, train_pairs, test_pairs):
    print(f"\n{'='*20} Запуск эксперимента: {exp_name} {'='*20}")

    #  Параметры (одинаковые для обоих экспериментов)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    INPUT_DIM = eng.n_words
    OUTPUT_DIM = rus.n_words
    ENC_EMB_DIM = 128
    DEC_EMB_DIM = 128
    HID_DIM = 256
    CLIP = 1
    N_EPOCHS = 5
    BATCH_SIZE = 128

    #  Загрузчики данных
    train_dataset = TranslationDataset(train_pairs, eng, rus)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

    #  Инициализация модели
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, n_layers, rnn_type)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, n_layers, rnn_type)
    model = Seq2Seq(enc, dec, device).to(device)

    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_token)

    #  Обучение
    for epoch in range(N_EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, CLIP)
        print(f'Эпоха: {epoch+1:02} | Потеря на обучении: {train_loss:.3f}')

    #  Оценка качества
    print("Вычисление BLEU score...")
    bleu = calculate_bleu(model, test_pairs, eng, rus, device)
    print(f"BLEU Score ({exp_name}): {bleu*100:.2f}")

    #  Примеры перевода
    print("\nПримеры перевода:")
    for i in range(3):
        src, trg = test_pairs[i]
        translation = translate_sentence(src, eng, rus, model, device)
        original = tokenize('ru', trg)
        print(f"Оригинал (EN): {src}")
        print(f"Эталон (RU):  {' '.join(original)}")
        print(f"Перевод (RU):  {' '.join(translation)}\n")

    return model, bleu

In [27]:
#  4. Запуск сравнения

# Эксперимент 1: GRU
model_gru, bleu_gru = run_experiment_eval(
    rnn_type='GRU',
    n_layers=1,
    exp_name="GRU 1 слой",
    train_pairs=train_pairs,
    test_pairs=test_pairs
)

# Эксперимент 2: LSTM
model_lstm, bleu_lstm = run_experiment_eval(
    rnn_type='LSTM',
    n_layers=1,
    exp_name="LSTM 1 слой",
    train_pairs=train_pairs,
    test_pairs=test_pairs
)


==================== Запуск эксперимента: GRU 1 слой ====================


Training: 100%|██████████| 782/782 [04:54<00:00,  2.65it/s]


Эпоха: 01 | Потеря на обучении: 4.812


Training: 100%|██████████| 782/782 [04:54<00:00,  2.66it/s]


Эпоха: 02 | Потеря на обучении: 3.392


...

Эпоха: 03 | Потеря на обучении: 3.174


Training: 100%|██████████| 782/782 [04:59<00:00,  2.61it/s]


Эпоха: 04 | Потеря на обучении: 2.637


Training: 100%|██████████| 782/782 [04:56<00:00,  2.64it/s]


Эпоха: 05 | Потеря на обучении: 2.223
Вычисление BLEU score...
BLEU Score (LSTM 1 слой): 31.18

Примеры перевода:
Оригинал (EN): Tom bought a used truck.
Эталон (RU):  Том купил подержанный грузовик .
Перевод (RU):  купил билет .

Оригинал (EN): We know who you are.
Эталон (RU):  Мы знаем , кто ты .
Перевод (RU):  знаю , кто ты .

Оригинал (EN): I'd like to go with Tom.
Эталон (RU):  Я хотел бы пойти с Томом .
Перевод (RU):  с с с .



In [28]:
#  5. Итоги сравнения
print("\n" + "="*50)
print("ИТОГИ СРАВНЕНИЯ КАЧЕСТВА:")
print(f"GRU BLEU Score: {bleu_gru*100:.2f}")
print(f"LSTM BLEU Score: {bleu_lstm*100:.2f}")
if bleu_lstm > bleu_gru:
    print("Победитель: LSTM (показал более высокое качество)")
elif bleu_gru > bleu_lstm:
    print("Победитель: GRU (показал более высокое качество)")
else:
    print("Ничья: модели показали одинаковое качество")
print("="*50)


ИТОГИ СРАВНЕНИЯ КАЧЕСТВА:
GRU BLEU Score: 37.21
LSTM BLEU Score: 31.18
Победитель: GRU (показал более высокое качество)


Анализ результатов
1. Потери на обучении (Training Loss)

GRU: 1.788 (последняя эпоха)
LSTM: 2.223 (последняя эпоха)
Вывод: GRU не только обучился быстрее (его потери падали стремительнее), но и достиг меньшего финального значения потерь. Это говорит о том, что на наших данных GRU-модель смогла лучше "подстроиться" под обучающую выборку.

2. BLEU Score (Объективное качество перевода)

GRU: 37.21
LSTM: 31.18
Вывод: Это самый важный результат. Higher is better. GRU показал значительно более высокое качество перевода на тестовых данных, которые модель никогда не видела. Разница в 6 пунктов BLEU — это существенная разница, которая подтверждает, что GRU в данном случае не просто лучше запомнила данные, а научилась делать более качественные переводы.

3. Примеры перевода

Здесь мы видим подтверждение цифр:

GRU делает более осмысленные, хоть и грамматически неверные, попытки. Например, хотел пойти с нами для I'd like to go with Tom — это довольно близко по смыслу.
LSTM ведет себя более нестабильно. Он может дать отличный перевод (знаю , кто ты .), а может и полную бессмыслицу (с с с .). Такой разброс в качестве говорит о том, что модели сложнее стабильно выдавать хороший результат.
Итоговый вывод
Победитель: GRU.

В нашем конкретном эксперименте, с заданными параметрами, объемом данных и временем обучения, GRU-ячейки оказались эффективнее LSTM-ячеек.

Почему так произошло?

Простота: У GRU меньше параметров (2 затвора против 3 у LSTM). На относительно простой задаче и не очень большой модели эта простота оказалась плюсом. Модели было легче и быстрее найти хорошие веса.
Скорость обучения: GRU обучается быстрее, и за 5 эпох он успел "научиться" больше, чем LSTM.
Переобучение: Более сложная LSTM-модель могла начать переобучаться или просто "запутаться" из-за своей сложности на ограниченном количестве данных и эпох.
Это отличный пример того, что теоретическое преимущество LSTM в обработке длинных последовательностей не всегда превращается в практический выигрыш на конкретной задаче.